# *Importing Data*

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import os
import shutil

In [3]:
def filter_classes(original_dir, target_dir, classes):
    """
    Filters and copies only the specified classes from the original dataset directory to a new directory.

    Parameters:
        original_dir (str): Path to the original dataset directory containing all classes.
        target_dir (str): Path to the target directory where filtered classes will be copied.
        classes (list): List of class names to include (e.g., ['cars', 'buses', 'trucks']).
    """
    if os.path.exists(target_dir):
        shutil.rmtree(target_dir)
    os.makedirs(target_dir)
    
    for class_name in classes:
        class_dir = os.path.join(original_dir, class_name)
        if os.path.exists(class_dir):
            shutil.copytree(class_dir, os.path.join(target_dir, class_name))
        else:
            print(f"Class directory {class_name} does not exist in {original_dir}. Skipping.")


In [5]:
def load_filtered_dataset(classes, train_dir, val_dir, test_dir, target_dir, img_size, batch_size):
    """
    Loads the dataset by filtering out only the specified classes from the dataset directories.

    Parameters:
        train_dir (str): Path to the original training dataset directory.
        val_dir (str): Path to the original validation dataset directory.
        test_dir (str): Path to the original test dataset directory.
        target_dir (str): Path to the directory where filtered classes will be copied.
        img_size (tuple): Target size for images (height, width).
        batch_size (int): Batch size for the data generator.

    Returns:
        train_generator, val_generator, test_generator: Data generators for the filtered dataset.
    """
    # Create filtered directories
    filtered_train_dir = os.path.join(target_dir, 'train')
    filtered_val_dir = os.path.join(target_dir, 'val')
    filtered_test_dir = os.path.join(target_dir, 'test')
    
    filter_classes(train_dir, filtered_train_dir, classes)
    filter_classes(val_dir, filtered_val_dir, classes)
    filter_classes(test_dir, filtered_test_dir, classes)
    
    # Load filtered dataset
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        filtered_train_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical'
    )

    val_generator = val_test_datagen.flow_from_directory(
        filtered_val_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical'
    )

    test_generator = val_test_datagen.flow_from_directory(
        filtered_test_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    return train_generator, val_generator, test_generator

In [12]:
# Classes to filter
classes = ['car', 'bus', 'truck']

TRAIN_DIR = 'dataset/train'
VAL_DIR = 'dataset/validation'
TEST_DIR = 'dataset/test'

# Target directory for filtered dataset
TARGET_DIR = 'filtered_dataset'

# Load only the specified classes
train_gen, val_gen, test_gen = load_filtered_dataset(classes, TRAIN_DIR, VAL_DIR, TEST_DIR, TARGET_DIR, IMG_SIZE, BATCH_SIZE)

Found 3032 images belonging to 3 classes.
Found 382 images belonging to 3 classes.
Found 387 images belonging to 3 classes.


# *Classification Model*

In [13]:
# Constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10
NUM_CLASSES = 3  # Cars, Buses, Trucks

In [14]:
# Function to create the classification model
def create_classification_model(input_shape=(224, 224, 3), num_classes=NUM_CLASSES):
    base_model = MobileNetV2(input_shape=input_shape, include_top=False)
    base_model.trainable = False  # Freeze the base model

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1024, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')  # Classification head
    ])
    
    return model

In [28]:
# Function to fine-tune and train the model
def train_model(model, train_generator, val_generator, epochs, batch_size):
    
    #checkpoint = ModelCheckpoint('ssd_vehicle_classification.weights.h5', monitor='val_loss', save_best_only=True)
    checkpoint = ModelCheckpoint('ssd_vehicle_classification.model.keras', monitor='val_loss', save_best_only=True)
    
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        validation_data=val_generator,
        validation_steps=val_generator.samples // batch_size,
        epochs=epochs,
        callbacks=[checkpoint]
    )
    
    return history

In [29]:
# Function to evaluate the model on the test dataset
def evaluate_model(model, test_generator):
    loss, accuracy = model.evaluate(test_generator)
    print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

In [31]:
model = create_classification_model(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), num_classes=NUM_CLASSES)
# Fine-tune and train the model
train_model(model, train_gen, val_gen, epochs=EPOCHS, batch_size=BATCH_SIZE)
# Evaluate the model on the test dataset
evaluate_model(model, test_gen)

Epoch 1/10


/Users/amrmohamed/Downloads/VLensTask/vlens/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


94/94 ━━━━━━━━━━━━━━━━━━━━ 32s 314ms/step - accuracy: 0.6904 - loss: 0.8837 - val_accuracy: 0.8778 - val_loss: 0.3142
Epoch 2/10
 1/94 ━━━━━━━━━━━━━━━━━━━━ 27s 300ms/step - accuracy: 0.8438 - loss: 0.3415

2024-09-04 22:35:37.940897: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/amrmohamed/anaconda3/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8438 - loss: 0.3415 - val_accuracy: 0.7333 - val_loss: 0.6237
Epoch 3/10


...

94/94 ━━━━━━━━━━━━━━━━━━━━ 31s 320ms/step - accuracy: 0.8228 - loss: 0.4382 - val_accuracy: 0.9006 - val_loss: 0.2886
Epoch 4/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9062 - loss: 0.2354 - val_accuracy: 0.7333 - val_loss: 0.4606
Epoch 5/10


2024-09-04 22:36:10.367048: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


94/94 ━━━━━━━━━━━━━━━━━━━━ 34s 342ms/step - accuracy: 0.8454 - loss: 0.4116 - val_accuracy: 0.8523 - val_loss: 0.3427
Epoch 6/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9062 - loss: 0.2526 - val_accuracy: 0.8667 - val_loss: 0.3032
Epoch 7/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 36s 372ms/step - accuracy: 0.8390 - loss: 0.3803 - val_accuracy: 0.8636 - val_loss: 0.3231
Epoch 8/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7812 - loss: 0.5016 - val_accuracy: 0.8000 - val_loss: 0.4052
Epoch 9/10


2024-09-04 22:37:21.656503: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


94/94 ━━━━━━━━━━━━━━━━━━━━ 36s 364ms/step - accuracy: 0.8394 - loss: 0.3915 - val_accuracy: 0.8693 - val_loss: 0.3185
Epoch 10/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8125 - loss: 0.3638 - val_accuracy: 0.9000 - val_loss: 0.3090
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 263ms/step - accuracy: 0.8814 - loss: 0.3013
Test Loss: 0.2927, Test Accuracy: 0.8837


In [ ]:
# Main script execution
if __name__ == "__main__":
    # Load and preprocess the dataset
    train_gen, val_gen, test_gen = load_filtered_dataset(classes, TRAIN_DIR, VAL_DIR, TEST_DIR, TARGET_DIR, IMG_SIZE, BATCH_SIZE)    
    # Create the SSD model
    ssd_model = create_ssd_model(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), num_classes=NUM_CLASSES)
    
    # Fine-tune and train the model
    train_model(ssd_model, train_gen, val_gen, epochs=EPOCHS, batch_size=BATCH_SIZE)
    
    # Evaluate the model on the test dataset
    evaluate_model(ssd_model, test_gen)


# *YOLO*

In [32]:
from ultralytics import YOLO

# Load a pre-trained YOLOv8 model
model = YOLO('yolov8n.pt')  # You can use other versions like yolov8s.pt, yolov8m.pt based on your need.

# Train the model on COCO dataset with specific categories
# coco.yaml already contains 80 classes, we'll focus on 'car', 'truck', and 'bus'

# Define a custom class mapping for COCO categories we need (class IDs for 'car', 'truck', 'bus')
classes_of_interest = [2, 5, 7]  # Class IDs: car=2, bus=5, truck=7

100%|█████████████████████████████████████| 6.25M/6.25M [00:51<00:00, 128kB/s]


In [33]:
# Finetune YOLOv8 on COCO dataset with vehicle categories (car, truck, bus)
results = model.train(data='coco128.yaml',  # path to your dataset config
                      epochs=50,           # number of epochs
                      imgsz=640,           # image size
                      batch=16,            # batch size
                      name='custom_yolov8',  # name of the model
                      classes=classes_of_interest)  # restrict classes to car, truck, bus

Ultralytics YOLOv8.2.89 🚀 Python-3.10.9 torch-2.4.1 CPU (Apple M1)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coco128.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=custom_yolov8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=[2, 5, 7], retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_widt

100%|█████████████████████████████████████| 6.66M/6.66M [00:14<00:00, 484kB/s]
Unzipping /Users/amrmohamed/Downloads/datasets/coco128.zip to /Users/amrmohame

Dataset download success ✅ (16.4s), saved to /Users/amrmohamed/Downloads/datasets




                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

train: Scanning /Users/amrmohamed/Downloads/datasets/coco128/labels/train2017.

...

      50/50         0G     0.8406      1.089     0.8272          6        640: 
                 Class     Images  Instances      Box(P          R      mAP50  

                   all        128         65      0.924      0.606      0.797      0.596



50 epochs completed in 3.511 hours.
Optimizer stripped from /Users/amrmohamed/Downloads/VLensTask/runs/detect/custom_yolov8/weights/last.pt, 6.5MB
Optimizer stripped from /Users/amrmohamed/Downloads/VLensTask/runs/detect/custom_yolov8/weights/best.pt, 6.5MB

Validating /Users/amrmohamed/Downloads/VLensTask/runs/detect/custom_yolov8/weights/best.pt...
Ultralytics YOLOv8.2.89 🚀 Python-3.10.9 torch-2.4.1 CPU (Apple M1)
Model summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  


                   all        128         65      0.823       0.71      0.827      0.619
                   car         12         46      0.757      0.587      0.667      0.368
                   bus          5          7      0.847      0.794      0.944      0.873
                 truck          5         12      0.866       0.75       0.87      0.618
Speed: 1.2ms preprocess, 142.2ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /Users/amrmohamed/Downloads/VLensTask/runs/detect/custom_yolov8


In [3]:
from ultralytics import YOLO
import cv2

def load_model(model_path='yolov8n.pt'):
    """
    Loads the YOLOv8 model.
    
    Args:
        model_path (str): Path to the trained YOLOv8 model.

    Returns:
        model: Loaded YOLO model.
    """
    model = YOLO(model_path)
    return model

def predict_image(model, image_path, save_path='result_image.jpg'):
    """
    Performs object detection on a single image.
    
    Args:
        model: Loaded YOLOv8 model.
        image_path (str): Path to the input image.
        save_path (str): Path to save the result image with bounding boxes.

    Returns:
        results: YOLO detection results.
    """
    # Load the image
    image = cv2.imread(image_path)
    
    # Perform inference
    results = model(image)
    
    # Save and display the result
    results[0].save(save_path)
    return results


In [36]:
def predict_video(model, video_path, output_path='output_video.mp4'):
    """
    Performs object detection on a video.

    Args:
        model: Loaded YOLOv8 model.
        video_path (str): Path to the input video.
        output_path (str): Path to save the output video with bounding boxes.

    Returns:
        None
    """
    # Capture video from file
    cap = cv2.VideoCapture(video_path)
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define video writer to save output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # codec
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Perform inference on each frame
        results = model(frame)

        # Get the annotated frame
        annotated_frame = results[0].plot()

        # Write the frame into the output video
        out.write(annotated_frame)
        
    # Release resources
    cap.release()
    out.release()


In [5]:
model = load_model('../runs/detect/custom_yolov8/weights/best.pt')  # or use your custom model


In [42]:
results = predict_image(model, '../dataset/test/bus/1.png', save_path='../result_image.jpg')



0: 640x640 1 bus, 141.1ms
Speed: 9.6ms preprocess, 141.1ms inference, 14.2ms postprocess per image at shape (1, 3, 640, 640)


In [43]:
predict_video(model, 'test.mp4', output_path='result_video.mp4')


0: 384x640 11 cars, 2 trucks, 132.3ms
Speed: 31.3ms preprocess, 132.3ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 2 trucks, 92.6ms
Speed: 3.5ms preprocess, 92.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 2 trucks, 119.4ms
Speed: 5.3ms preprocess, 119.4ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 1 truck, 80.2ms
Speed: 3.4ms preprocess, 80.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 cars, 1 truck, 76.1ms
Speed: 3.8ms preprocess, 76.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 1 truck, 99.1ms
Speed: 4.8ms preprocess, 99.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 2 trucks, 110.1ms
Speed: 3.6ms preprocess, 110.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 2 trucks, 115.8ms
Speed: 4.1m

In [15]:
model.export(format='onnx')

Ultralytics YOLOv8.2.89 🚀 Python-3.10.9 torch-2.4.1 CPU (Apple M1)
Model summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from '../runs/detect/custom_yolov8/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

ONNX: starting export with onnx 1.16.2 opset 19...
ONNX: export success ✅ 0.5s, saved as '../runs/detect/custom_yolov8/weights/best.onnx' (12.2 MB)

Export complete (1.1s)
Results saved to /Users/amrmohamed/Downloads/VLensTask/runs/detect/custom_yolov8/weights
Predict:         yolo predict task=detect model=../runs/detect/custom_yolov8/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=../runs/detect/custom_yolov8/weights/best.onnx imgsz=640 data=/Users/amrmohamed/Downloads/VLensTask/vlens/lib/python3.10/site-packages/ultralytics/cfg/datasets/coco128.yaml  
Visualize:       https://netron.app


'../runs/detect/custom_yolov8/weights/best.onnx'

In [16]:
import onnx
onnx_model = onnx.load('../runs/detect/custom_yolov8/weights/best.onnx')
onnx.checker.check_model(onnx_model) 

In [17]:
import onnxruntime as ort

def print_model_input_names(onnx_model_path):
    # Load the ONNX model
    session = ort.InferenceSession(onnx_model_path)
    
    # Print input names
    input_names = [input.name for input in session.get_inputs()]
    print("Model Input Names:", input_names)
    
    return input_names


In [18]:
print_model_input_names('../runs/detect/custom_yolov8/weights/best.onnx')  # Replace with your model path


Model Input Names: ['images']


['images']